<a href="https://colab.research.google.com/github/Jacobmunoz88/se-ales-y-sistemas/blob/main/Parcial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Parcial #2 de Señales y Sistemas.**


# Solucion Problemas de simulascion y echos en un dashboard

In [1]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.4 MB/s eta 0:00:00


In [2]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile #librerias descarga Youtube y manejo de audios en python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━

In [3]:
token = '2lLjMZza0oSVL6rk1HPGahn6C8a_3tBJpBFke5DPuFyWDJzkA' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok

In [5]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run detector.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://bb1d-35-227-5-236.ngrok-free.app


In [4]:
%%writefile detector.py

import streamlit as st
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from pyngrok import ngrok
import yt_dlp as youtube_dl
import joblib
from scipy.fft import fft, fftfreq
from IPython.display import Audio
from scipy.io.wavfile import write
from matplotlib import animation, rc
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

st.markdown("# Parcial 2 ")
#widgets y funcionalidades de streamlit


#funcion para descargar mp3 desde youtube
def download_ytvid_as_mp3(custom_url,name):
    #video_url = input("enter url of youtube video:")
    video_info = youtube_dl.YoutubeDL().extract_info(url = custom_url,download=False)
    filename = f"{name}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }

    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    print("Download complete... {}".format(filename))

@st.cache_resource
def load_and_process_audio_files(path='results/'):
    # Function to train the model from the Excel file
    pass

def pagina1():
  st.title("Solucion prengunta 2")
  st.write("Realice las simulaciones respectivas para graficar el espectro de Fourier de la siguiente (magnitud y fase como diagrama de Bode en decibelios), y presente el error relativo y la senal reconstruida para $N = {1,2,..,50}$")
  st.latex(r"x\left( t \right)=\left| Asin(2\pi\ F_{o}t) \right|^{2}")
  st.write(r"Con $t \in  \left[- \frac{1}{2F_{o}}, \frac{1}{2F_{o}} \right]$ con $A, F_{o} \in \mathbb{\mathbb{R^{+}}}$")

  #simulamos la señal de entrada
  A = 4
  Fo = 6 #frecuencia fundamental
  ti = -1/(2*Fo) #tiempo inicial
  tf = 1/(2*Fo) #tiempo final
  T = tf-ti
  wo = 2*np.pi/T #frecuencia angular
  print('wo=',wo)
  print('Fo=',Fo)
  Fs = 30*Fo #frecuencia muestreo -> debe respetar Nyquist Fs >= 2 Fo
  tv = np.arange(ti,tf,1/Fs) #vector de tiempo
  Nm = len(tv)#muestras en el tiempo
  xo = abs(A*np.sin(2*np.pi*Fo*tv))**2 #señal de entrada




  fig1, ax1 = plt.subplots()
  st.write("# Grafica de la señal")
  ax1.plot(tv,xo) #gráfica de la señal
  ax1.set_xlabel('tiempo [s]')
  ax1.set_ylabel('$x(t)$')
  st.pyplot(fig1)


  #se presenta la simulación del espectro y la reconstrucción
  #construir bases serie exponencial
  N = 50 #numero armonicos
  wo = 2*np.pi/T # frecuencia fundamental

  # definir bases
  phin = np.zeros((Nm,2*N+1),dtype=np.complex_)#crear matriz para guardar bases
  for n in range(-N,N+1,1):
      phin[:,n+N] = np.exp(1j*n*wo*tv)#base de fourier en el intervalo de interés

  from ipywidgets import interact,IntSlider

  Nss = st.slider("Selecciona el N", min_value=1, max_value=50, value=5)
  fig2, ax2 = plt.subplots()
  ax2.plot(tv,np.real(phin[:,int(1+Nss)]),label="Re{$\phi_n$(t)}")
  ax2.set_xlabel("t[s]",fontsize = 14)
  ax2.set_ylabel("$\phi_n$(t)",fontsize = 14)
  ax2.plot(tv,np.imag(phin[:,int(1+Nss)]),label="Im{$\phi_n$(t)}")
  ax2.grid()
  ax2.legend()
  st.pyplot(fig2)


  #espectro de la señal - serie exponencial
  #calcular espectro -> cambiar según la señal estudiada
  Cn = np.zeros(2*N+1,dtype=np.complex_)
  nv = np.linspace(-N, N, 2*N+1) #vector num armónicos
  Cn[N] = (A**2)/2 # nivel dc C_0
  Cn[N-2] = -(A**2)/4
  Cn[N+2] = -(A**2)/4

  st.write("# Graficar espectro")
  #graficar espectro
  fig = plt.figure(figsize=(8,8))
  plt.subplot(2,2,1)
  plt.stem(nv, np.real(Cn), 'r')
  plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
  plt.ylabel(r'$Re\{C_n\}$',fontsize = 14)
  plt.axis('tight')


  plt.subplot(2,2,2)
  plt.stem(nv, np.imag(Cn), 'r')
  plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
  plt.ylabel(r'$Im\{C_n\}$',fontsize = 14)
  plt.axis('tight')


  plt.subplot(2,2,3)
  plt.stem(nv, abs(Cn), 'r')#magnitud del espectro
  plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
  plt.ylabel(r'$|C_n|$',fontsize = 14)
  plt.axis('tight')


  plt.subplot(2,2,4)
  plt.stem(nv, np.angle(Cn), 'r')#fase del espectro
  plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
  plt.ylabel(r'$\langle C_n$',fontsize = 14)
  plt.axis('tight')


  st.pyplot(plt)


  st.write("# Señal reconstruida")
  #reconstrucción
  Px = (3*A**4)/8
  #graficar reconstrucción
  Ns = st.slider("Selecciona valor del N", min_value=1, max_value=50)
  ind = range(Ns-1,Ns+2)
  er = 1- np.sum(abs(Cn[ind])**2)/Px
  #señal reconstruida
  xe = phin[:,ind].dot(Cn[ind] ) # dot calcula el producto entre vectores
  fig3, ax3 = plt.subplots()
  ax3.plot(tv,xe,color='b',label='q') # señal estimada o filtrada
  ax3.plot(tv,xo,color='r',label='q') #señal original
  ax3.set_title('E_r=%.2f[%%]' % (100*er),fontsize=16)
  ax3.set_xlabel('t[s]')
  ax3.set_ylabel('x(t)')
  ax3.grid()
  ax3.legend()
  st.pyplot(fig3)

  xofft= np.fft.fft(xo)
  ftftt= np.fft.fftfreq(len(xo),1/Fs)
  fft_magnitud_db = 20 * np.log10(xofft)
  fft_fase = np.angle(xofft)
  fig4, (ax5, ax4) = plt.subplots(2, 1, figsize=(8, 6))
  # Magnitud (en decibelios)
  ax5.semilogx(ftftt, fft_magnitud_db)
  ax5.set_title("Diagrama de Bode - Magnitud (FFT)")
  ax5.set_xlabel("Frecuencia [Hz]")
  ax5.set_ylabel("Magnitud [dB]")
  ax5.grid(True)

  # Fase (en grados)
  ax4.semilogx(ftftt, np.degrees(fft_fase))
  ax4.set_title("Diagrama de Bode - Fase (FFT)")
  ax4.set_xlabel("Frecuencia [Hz]")
  ax4.set_ylabel("Fase [grados]")
  ax4.grid(True)

  # Mostrar el gráfico en Streamlit
  st.pyplot(fig4)


def pagina2():
  st.title("Solucion pregunta 3")
  st.title("Graficador y Reproductor de la modulacion de una cancion de youtube")
  st.write("Se necesita un archivo execel con las siguientes caracteristicas; link de la cancion, la banda, y el genero de la cancion.")
  st.write("Solo se capturara del segundo 20 al 25 de la cancion escogida, donde se le hallara las grafica de las señales en tiempo y frecuencia (magnitud y fase) de la señal mensaje, portadora y modulada y se reproduzca los fragmentos de audio del mensaje, portadora y señal modulada.")
  uploaded_file = st.file_uploader("Upload your Excel file", type=["xlsx"])

  if uploaded_file:
      df = pd.read_excel(uploaded_file)
      st.write("Uploaded data:", df)

      try:
        os.mkdir('results')
      except:
        print("Carpeta results ya existe")

      #recorrer excel con videos
      N, P = df.shape
      Ns = N * 5 #cantidad de segmentos por cancion
      n = 0
      print(f"video {n+1} de {N}")
      print(f"link: {df.loc[n,'link']}\n")
      print(f"band: {df.loc[n,'band']}\n")
      print(f"type: {df.loc[n,'type']}\n")
      #ruta video n-th
      name_ = 'results/'+df.loc[n,'band']+"_"+str(n)+"_"+str(df.loc[n,'type_num']) # #video+nombre+tipo de genero musical
      #descargar mp3 desde youtube
      can = download_ytvid_as_mp3(df.loc[n,'link'],name_)
      #convertir a .wav
      subprocess.call(['ffmpeg','-y', '-i', name_+'.mp3',
                      name_+'.wav'])


      #cargar .wavs y partir audios
      #lista archivos .wav

      path = 'results/'
      wav_files = [f for f in os.listdir(path) if f.endswith('.wav')]


      #leer archivos y crear np.array audios
      fs = 48000
      tl = np.array([20])
      ts = 5 #t segmento
      Ns = len(wav_files)*len(tl) #cantidad segmentos
      x_t = np.zeros((Ns,int(ts*fs),2)) #Ns segmentos, cantidad de muestras, 2 canales (stereo)
      label = np.zeros((Ns,1)) #vector tipo de genero
      name_c = []
      #leer archivos wav
      i = 0
      for name in wav_files:#lectura audio .wav
          x, fs = sf.read(path+name)
          for ti in tl: #segmentos de tiempo
              x_t[i] = x[int(fs*ti):int(fs*(ti+ts)),:]
              label[i] = int(name[-5]) #tipo de genero
              name_c += [name[:-6]]
              print(f"{i} lectura: {name}; segundo {ti}:{ti+ts}; tipo música {label[i]}")
              i+=1

      st.audio(name_+'.mp3')

      st.write("Fragmento del audio")

      aa= x_t[0]
      write("aa.wav", fs, aa.astype(np.float32))

      st.audio("aa.wav")

      #tomamos canal 1 para generar la modulación
      xm = x_t[0,:,0]

      Fc = 15000 #frecuencia de portadora
      t = np.arange(0,ts,1/fs) #vector del tiempo
      Im = st.number_input("Elija un indice de modulacion:", )  # Im = peak(m(t))/Ac ->  Ac = peak(m(t))/Im
      if Im:
        Ac = max(abs(xm))/Im #amplitud de la portadora según índice de modulación
        c = Ac*np.sin(2*np.pi*Fc*t) #portadora
        print(Ac)

        y = (1+xm/Ac)*c #señal modulada en AM


    #calculo de espectros
        Xfc = np.fft.fft(c)
        Xfm = np.fft.fft(xm)
        Xfy = np.fft.fft(y)

        vfre = np.fft.fftfreq(len(c),1/fs)



        st.write("### Selecciona el gráfico:")
        option = st.selectbox("Gráfico", ["portadora y del mensaje", "modulada en AM", "espectro"])
        # Graficar según la opción seleccionada
        if option == "portadora y del mensaje":
            st.write('# Grafica de la señal portadora y del mensaje')
            plt.plot(t,c,label='portadora')
            plt.plot(t,xm,label='mensaje')
            plt.xlabel('t [s]')
            plt.ylabel('Amplitud')
            plt.legend()
            st.pyplot(plt)

            #reproducir portadora
            write("c.wav", fs, c.astype(np.float32))

            st.audio("c.wav")

        elif option == "modulada en AM":
            st.write('# Señal modulada en AM')
            plt.plot(t,y)
            plt.xlabel('$t[s]$')
            plt.ylabel('$y(t)$')
            st.pyplot(plt)

            #reproducir modulada
            write("y.wav", fs, y.astype(np.float32))

            st.audio("y.wav")

        elif option == "espectro":
            st.write("# Señal del espectro")
            plt.plot(vfre,abs(Xfy),label='modulada')
            plt.plot(vfre,abs(Xfm),label='mensaje')
            plt.plot(vfre,abs(Xfc),label='portadora')
            plt.legend()
            plt.xlabel('$f [Hz]$')
            plt.ylabel('$|X(f)|$')
            plt.ylim([min(abs(Xfm)), 1.1*max(abs(Xfm))])
            st.pyplot(plt)

def pagina3():
  st.title("Solucion pregunta 4")

  st.write("# Distorsión Armónica Total (Total Harmonic Distortion - THD)")
  st.write("¿Qué es la THD? La distorsión armónica total (THD) es una medida que cuantifica la presencia de armónicos en una señal eléctrica o de voltaje. Los armónicos son componentes de frecuencia múltiplo de la frecuencia fundamental que distorsionan la forma de onda original, generalmente sinusal. El THD se expresa comúnmente como un porcentaje de la amplitud de la señal fundamental.")
  st.latex(r"THD=\frac{\sqrt{V_{2}^{2}+V_{3}^{2}+...+V_{n}^{2}}}{V_{1}}")
  st.write("Donde:")
  st.write("$V_1$ es la amplitud de la componente fundamental.")
  st.write("$V_2, V_3, \dots, V_n$ son las amplitudes de los armónicos de orden 2, 3, ... n")
  st.write("THD desde la FFT: La transformada rápida de Fourier (FFT) se utiliza para descomponer una señal en sus componentes frecuenciales, incluida la fundamental y sus armónicos. El THD se puede calcular identificando la amplitud de la componente fundamental y las amplitudes de los armónicos en el espectro de frecuencia obtenido por la FFT, y luego aplicando la fórmula mencionada.")
  st.write("# Factor de Potencia y su Relación con el THD")
  st.write("¿Qué es el Factor de Potencia? El factor de potencia es una medida de cuán eficientemente se utiliza la potencia eléctrica. Es la relación entre la potencia activa (que realiza un trabajo útil) y la potencia aparente (que es la combinación de la potencia activa y la potencia reactiva).")
  st.write("El factor de potencia también puede verse afectado por la distorsión armónica. El factor de potencia total ($PF_{total}$) se puede calcular considerando tanto el desplazamiento de la fase ($PF_{desplazamiento}$) como la distorsión armónica (THD):")
  st.latex(r"PF_{total}=PF_{Desplazamiento} \ast \frac{1}{\sqrt{1 + THD^{2}}}")
  st.write("# Ejemplo(Simulacion)")
  st.write("Configuracion")
  st.write("Frecuencia de la fuente: 60 Hz")
  st.write("Frecuencia de muestreo: 180 Hz")
  st.write("Amplitud de la fuente: 120")
  st.write("Periodo de la señal: 1/60")
  st.write("Periodo de muestreo: 1/180")
  # Parámetros de la simulación
  Vm = 120  # Amplitud de la fuente (V)
  f = 60    # Frecuencia de la fuente AC (Hz)
  R = st.number_input("Valor de resistencia:", )  # Resistencia en ohmios
  C = st.number_input("Valor de capacitor:", ) # Capacitancia en faradios
  To = 1/f   # Periodo de la señal
  Fs = 30*f
  Ts = 1/Fs
  t = np.arange(0, 5*To, Ts)  # Tiempo para dos ciclos

  # Fuente AC
  V_ac = Vm * np.sin(2 * np.pi * f * t)

  # Respuesta del rectificador de onda completa con carga resistiva pura
  V_rect_R = np.abs(V_ac)
  if R and C:
    # Respuesta del rectificador de onda completa con carga RC en serie
    # Cálculo de la corriente y tensión en el capacitor
    omega = 2 * np.pi * f
    Z_C = 1 / (1j * omega * C)  # Impedancia capacitiva
    Z_total = R + Z_C
    I_RC = V_rect_R / np.abs(Z_total)
    V_C = I_RC * np.abs(Z_C)
    V_rect_RC = np.abs(V_C)  # Tensión en el capacitor


    # FFT para el caso resistivo puro
    N = len(t)
    V_rect_R_fft = np.fft.rfft(V_rect_R)
    freqs = np.fft.rfftfreq(N, Ts)

    # FFT para el caso RC en serie
    V_rect_RC_fft = np.fft.rfft(V_rect_RC)

    # Cálculo del THD para carga resistiva pura
    V1_R = np.abs(V_rect_R_fft[1]) / N
    harmonics_R = np.abs(V_rect_R_fft[2:N//2]) / N
    THD_R = np.sqrt(np.sum(harmonics_R**2)) / V1_R

    # Cálculo del THD para carga RC en serie
    V1_RC = np.abs(V_rect_RC_fft[1]) / N
    harmonics_RC = np.abs(V_rect_RC_fft[2:N//2]) / N
    THD_RC = np.sqrt(np.sum(harmonics_RC**2)) / V1_RC

    st.write("# Señal de alimentación")
    plt.plot(t,V_ac,label='Alimentación')
    plt.legend()
    st.pyplot(plt)

    st.write("# Señales rectificadas de entrada y salidas ")
    plt.plot(t, V_C,label='In(t)')#entrada rectificada
    plt.plot(t, V_rect_RC,label='Out(t)')#salida del circuito
    plt.xlabel('t [s]')
    plt.ylabel('out(t)')
    st.pyplot(plt)

    st.write("# Señal del espectro")
    plt.stem(freqs,abs(V_rect_RC_fft/len(V_rect_RC)))
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('f')
    st.pyplot(plt)

    st.write(f"Es es el valor del THD para carga resistiva pura: ¨{THD_R}")
    st.write(f"Es es el valor del THD para carga RC: ¨{THD_RC}")

    PF_R= 1/np.sqrt(1+THD_R**2)
    PF_RC = 1/np.sqrt(1+THD_RC**2)

    st.write(f"Es es el valor del PF para carga resistiva pura: ¨{PF_R}")
    st.write(f"Es es el valor del PF para carga RC: ¨{PF_RC}")





# Crear un diccionario de páginas para facilitar la navegación
paginas = {
    "Página 1": pagina1,
    "Página 2": pagina2, "Página 3": pagina3}

# Barra lateral para seleccionar la página
st.sidebar.title("Navegación")
opcion = st.sidebar.radio("Selecciona una página", list(paginas.keys()))

# Mostrar el contenido de la página seleccionada
paginas[opcion]()

Writing detector.py
